In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc6.dev101
cdv, version 1.1.2
Python 3.10.8


- [Standard Transactions](https://chialisp.com/standard-transactions)
- [p2_delegated_puzzle_or_hidden_puzzle.py](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py)
- [p2_delegated_puzzle_or_hidden_puzzle.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.clvm)

```clojure
; A puzzle should commit to `SYNTHETIC_PUBLIC_KEY` (curried)
(SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle solution)
```

```clojure
    ; conditions = (a delegated_puzzle solution))

      (if original_public_key
          ; hidden puzzle
          (assert
              (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
              conditions
          )
          ; delegated puzzle
          (c (list AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)) conditions)
      )
```

Two Paths:
1. Hidden puzzle path: you can solve the hidden puzzle by revealing the `original_public_key`, the hidden puzzle in `delegated_puzzle`, and a solution to the hidden puzzle.

> The hidden puzzle functionality is not used by the official wallet. It is there to allow for future functionality as needed.

> If the hidden puzzle path is taken, the `hidden puzzle` and `original public key` will be revealed
which proves that it was hidden there in the first place.

```clojure
    ; "is_hidden_puzzle_correct" returns true iff the hidden puzzle is correctly encoded

    (defun-inline is_hidden_puzzle_correct (SYNTHETIC_PUBLIC_KEY original_public_key hidden_puzzle)
      (=
          SYNTHETIC_PUBLIC_KEY
          (point_add
              original_public_key
              (pubkey_for_exp (sha256 original_public_key (sha256tree1 hidden_puzzle)))
          )
      )
    )

    ...
    (assert
      (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
      conditions
    )
```

> This roughly corresponds to bitcoin's taproot.


2. Delegated puzzle path: pass in 0 or `()` for `original_public_key` if it wants to use an arbitrary `delegated_puzzle` (and `solution`) signed by the `SYNTHETIC_PUBLIC_KEY` (whose corresponding private key can be calculated if you know the private key for `original_public_key`).

> a delegated puzzle, as in "graftroot", which should return the desired conditions.
```python
    # graftroot, a puzzle returns a list of conditions
    delegated_puzzle_solution = Program.to((1, condition_args))
    # we have a list of conditions, solution is ignored
    # conditions = (a delegated_puzzle solution))
    solutions = Program.to([[], delgated_puzzle_solution, []])  
```

```clojure
    ; prepend AGG_SIG_ME to the list of conditions
    (c (list AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)) conditions)
```

## Hiding Hidden Puzzle

> We also want the ability to pre-commit to a puzzle without revealing it.

> If this hidden puzzle were to be curried it in, any spend (even the delegated spend case) would reveal the full puzzle including the hidden puzzle.

> ...we wouldn't be able to lock up a coin with the same puzzle anymore, or people would be able to tell that the puzzle hash is the same and spend it without our consent.

> We can attempt to solve this problem by hashing the hidden puzzle. However, this has a similar problem in that if you spend the hidden case even once, people could see any identical puzzle hashes later and spend them without your consent.

> We need the puzzle to be hidden, but also have some entropy that keeps it unique to us.

In [2]:
original_sk = PrivateKey.from_bytes(bytes.fromhex("31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9"))
original_pk = original_sk.get_g1()
print(f" original sk: {bytes(original_sk).hex()}")
print(f" original pk: {original_pk}")

 original sk: 31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9
 original pk: b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84


### Synthentic keys
> The solution that the standard transaction uses is to **derive a new private key from the hidden puzzle and the original public key that can sign for the delegated spend case**.

> This is known as the `synthetic_offset`:

`synthetic_offset = sha256(original_public_key + hidden_puzzle_hash);`

> We then calculate the public key of this new private key (i.e. `synthetic_offset`), and add it to our existing original public key.

> This is known as the `synthetic_public_key`:

`synthentic_public_key = original_public_key + synthetic_offset_pubkey`

In [3]:
# sha256 of original pk and hidden puzzle hash
synthetic_offset = calculate_synthetic_offset(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)
synthetic_offset_sk = PrivateKey.from_bytes(synthetic_offset.to_bytes(32, "big"))
synthetic_offset_pk = synthetic_offset_sk.get_g1()

# Commit to Puzzle
synthetic_pk = original_pk + synthetic_offset_pk

print(f"synthetic_offset_sk:\t{bytes(synthetic_offset_sk).hex()}")
print(f"synthetic_offset_pk:\t{synthetic_offset_pk}")
print(f"synthetic_pk:\t\t{synthetic_pk}")

calculated_synthetic_pk = calculate_synthetic_public_key(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)

assert synthetic_pk == calculated_synthetic_pk

synthetic_offset_sk:	6d09189e26a65fd35f4ba150958489f696b85fc87cb7bb665b65f61e6fb23a80
synthetic_offset_pk:	b8aeaa286ce9d6fb34eac9228f6effc6b8692e470ea4316489bfce016fab3dd3f6954352c444b553254dbeb40fc25bf5
synthetic_pk:		8410a8a862f025d72b90ec667cd2a47dd1f8fe8667b14ff2cd90b62c104ba27acea8785ef3032db616eadad00069d298


### `synthentic_public_key = original_public_key + synthetic_offset_pubkey`
> You may wonder why we add the public key from our derived private key to the original public key when it's already part of the derivation. **This is because we use the synthetic public key to verify the signature of our delegated spends as well**.

```clojure
AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle))
```

> When you add two public keys, the sum of their private keys gives the private key for the resulting public key. If we didn't add the original public key then anyone who knew the hidden puzzle could derive the synthetic private key and could then perform delegated spends! Adding the original public key ensures that there is still a secret component of the synthetic private key, even though half of it can be known.

```python
# sum of pks
synthetic_pk = original_pk + synthetic_offset_pk

# sum of sks
synthetic_sk = original_sk + synthetic_offset_sk

# pk of sum of sks = sum of pks
original_pk + synthetic_offset_pk = synthetic_sk.get_g1()
```

> This secret component is the **private key** for the original public key.

You don't know `synthetic_sk` unless you know `original_sk`

In [4]:
calcualted_synthetic_sk = calculate_synthetic_secret_key(original_sk, DEFAULT_HIDDEN_PUZZLE_HASH)
synthetic_sk = PrivateKey.aggregate([original_sk, synthetic_offset_sk])

print(f"original_sk + synthetic_offset_sk\n   or\nsynthetic_sk\n {bytes(synthetic_sk).hex()}")

assert synthetic_sk == calcualted_synthetic_sk
assert synthetic_pk == synthetic_sk.get_g1()
assert original_pk + synthetic_offset_pk == synthetic_sk.get_g1()

original_sk + synthetic_offset_sk
   or
synthetic_sk
 2af3ee247c10b43f6c75ee8ea390795b97cfb7dce70edde4e2c6a6c137a28b78


### Spend
> This technique is also neat because it allows us to hide the **hidden puzzle** in a piece of information that was already necessary for the delegated spend. It's impossible to guess what the hidden puzzle is (because of original_pk), even if it's a standard hidden puzzle. It's even hard to tell if there's a hidden puzzle at all. All of this can contribute to the overall privacy.

> For example, if two parties agree to lock up some coins with a hidden puzzle together, you can share pubkeys and verify that information on the blockchain without revealing anything to the network. 

> Then, if you both agree that the coins can be spent with the hidden puzzle if either party is dishonest, you can trustlessly delegated spend the coins to the correct destinations and it's impossible to tell that they are not just normal everyday spends.

When a coin is spent, `puzzle` and `original_pk`, `hidden_puzzle`, and `solution_for_hidden_puzzle` are known.

The coin with the same puzzle hash can be spent the same way.

```python
solution = Program.to([original_pk, hidden_puzzle, solution_for_hidden_puzzle])  
```

```clojure
; hidden puzzle
; conditions = (a hidden_puzzle solution_for_hidden_puzzle))
; conditions should include AGG_SIG_ME
(assert
  (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
  conditions
)
```


### Spend Hidden Puzzle

In [41]:
message = "hidden puzzle message"
hidden_puzzle = Program(
    compile_clvm_text(
'''
(mod ()
    (include condition_codes.clib)
    (list
        (list CREATE_COIN_ANNOUNCEMENT (sha256 {message}))
    )
)
'''.format(message=message), search_paths=["/Users/karlkim/kimsk/chia-concepts/shared"]
    )
)

print_program(hidden_puzzle)
print_program(hidden_puzzle.run([]))

(a (q 4 (c 2 (q 0x0857fa0746661ee8531b0316bf71f5e0ef5323a03b4d6ad60964cf71d74dc37a)) ()) (c (q . 60) 1))
((60 0x0857fa0746661ee8531b0316bf71f5e0ef5323a03b4d6ad60964cf71d74dc37a))


In [42]:
synthetic_pk = calculate_synthetic_public_key(original_pk, hidden_puzzle.get_tree_hash())

standard_hidden_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_synthetic_public_key(synthetic_pk)
standard_hidden_ph = standard_hidden_puzzle.get_tree_hash()
print(standard_hidden_ph)

eff6b61b3e30877d4abab459afb714def8e51dc0377e7dfd8d7deffb30312d77


In [43]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")
coin = Coin(
    parent_coin_info,
    standard_hidden_ph,
    1
)

solution = Program.to([original_pk, hidden_puzzle, []])

coin_spend = CoinSpend(
    coin,
    standard_hidden_puzzle,
    solution
)

synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    original_sk,
    standard_hidden_ph
)

spend_bundle = SpendBundle(
    [coin_spend],
    G2Element()
)

print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_spends": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0xeff6b61b3e30877d4abab459afb714def8e51dc0377e7dfd8d7deffb30312d77"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bfff

### Spend Hidden Puzzle with AGG_SIG_ME

In [53]:
message = "hidden puzzle message"
hidden_puzzle = Program(
    compile_clvm_text(
'''
(mod (PK)
    (include condition_codes.clib)
    (list
        (list AGG_SIG_ME PK (sha256 "{message}"))
    )
)
'''.format(message=message), search_paths=["/Users/karlkim/kimsk/chia-concepts/shared"]
    )
).curry(original_pk)

print_program(hidden_puzzle)
print_program(hidden_puzzle.run([]))

(a (q 2 (q 4 (c 2 (c 5 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))) ()) (c (q . 50) 1)) (c (q . 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84) 1))
((50 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))


In [54]:
synthetic_pk = calculate_synthetic_public_key(original_pk, hidden_puzzle.get_tree_hash())

standard_hidden_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_synthetic_public_key(synthetic_pk)
standard_hidden_ph = standard_hidden_puzzle.get_tree_hash()
print(standard_hidden_ph)

07389aba93b611096a35e82141824243f5f3708aee09e2158d2115dd8394c8ba


In [55]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")
coin = Coin(
    parent_coin_info,
    standard_hidden_ph,
    1
)

solution = Program.to([original_pk, hidden_puzzle, []])

coin_spend = CoinSpend(
    coin,
    standard_hidden_puzzle,
    solution
)

sig =  AugSchemeMPL.sign(original_sk,
    (
        std_hash(bytes(message, "utf-8"))
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

assert AugSchemeMPL.verify(
    original_pk, 
    (
        std_hash(bytes(message, "utf-8"))
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    ),
    sig)

spend_bundle = SpendBundle(
    [coin_spend],
    sig
)

print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xa4eeafa23a6d9a7e9f941fb6a7cfe55407f291bacb0428cef2b99dd14f7cfc580626072c163b1b83bafdbeed3517602f197d996da04cc8bec52d59bb3e51e1e5bf94fe8726adc5d86107a6855fadb2295ad722e8e74cb473522991cb082c2d22",
    "coin_spends": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x07389aba93b611096a35e82141824243f5f3708aee09e2158d2115dd8394c8ba"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bfff

### Spend Delegated Puzzle

In [33]:
synthetic_pk = calculate_synthetic_public_key(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)
standard_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_synthetic_public_key(synthetic_pk)
standard_delegated_ph = standard_delegated_puzzle.get_tree_hash()
print(standard_delegated_ph)

879f4dc5ec5d30c6b6aec9a8531d2ec209bd618da967a4a462dececf9184c767


In [35]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")
coin = Coin(
    parent_coin_info,
    standard_delegated_ph,
    1
)

conditions = [
    [
        ConditionOpcode.CREATE_COIN_ANNOUNCEMENT,
        std_hash(bytes(message, "utf-8"))
    ]
]

delegated_puzzle: Program = puzzle_for_conditions(conditions) 
solution = solution_for_conditions(conditions)

coin_spend = CoinSpend(
    coin,
    standard_delegated_puzzle,
    solution
)

    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    original_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

sig =  AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

spend_bundle = SpendBundle(
    [coin_spend],
    sig
)

print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x865040e3d472ae25c80fb61a98395b0c41d948e942b944f05b202c8e447973d26ef6ad6f0cdc133f35a585740182ec85167d8404f02ce29c053647d26bd86cf960156f507911a778457c226783cd157744182d6dfe458c72f351a654d3f56dfe",
    "coin_spends": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x879f4dc5ec5d30c6b6aec9a8531d2ec209bd618da967a4a462dececf9184c767"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bfff